# 3) Reconstruct target file with source collection frames

This is the final notebook of the Freesound AMPLAB session and contains the code that performs *audio mosaicing* to construct a new version of the target file by using audio frames chosen from the source collection. This notebook used the DataFrames generated in the previous notebooks which contain metadata about the Freesound sounds in the source collection, the analysis results of the source collection and the analysis results of the target audio file.

In [ ]:
'''
# Essentia
!pip install essentia
# Freesound-python
!pip install git+https://github.com/mtg/freesound-python.git
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/SMC/AMPLab2324/AMPLAB 2024 Freesound session'
'''

In [2]:
import os
import pandas as pd
import essentia
import essentia.standard as estd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from IPython.display import display, Audio

In [13]:
# Load all DataFrames created in the previous notebooks
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
LEAD_SOUNDS_DIR = 'files/lead_sounds'  # Place where to store the lead sounds. Will be relative to the current folder.
LEAD_SOUNDS_METADATA_FILENAME = 'lead_sounds_metadata.csv'  # File where we'll store the metadata of our lead sounds collection
BASS_SOUNDS_DIR = 'files/bass_sounds'  # Place where to store the bass sounds. Will be relative to the current folder.
BASS_SOUNDS_METADATA_FILENAME = 'bass_sounds_metadata.csv'  # File where we'll store the metadata of our bass sounds collection
AMB_SOUNDS_DIR = 'files/amb_sounds'  # Place where to store the amb sounds. Will be relative to the current folder.
AMB_SOUNDS_METADATA_FILENAME = 'amb_sounds_metadata.csv'  # File where we'll store the metadata of our amb sounds collection

LEAD_DATAFRAME_SOURCE_FILENAME = 'lead_dataframe_source.csv'  # DataFrame file where to store the results of our analysis
BASS_DATAFRAME_SOURCE_FILENAME = 'bass_dataframe_source.csv'  # DataFrame file where to store the results of our analysis
AMB_DATAFRAME_SOURCE_FILENAME = 'amb_dataframe_source.csv'  # DataFrame file where to store the results of our analysis

DATAFRAME_TARGET_FILE_FILENAME = 'dataframe_target.csv'  # DataFrame file where to store the results of our analysis

df_lead = pd.read_csv(open(LEAD_SOUNDS_METADATA_FILENAME), index_col=0)
df_bass = pd.read_csv(open(BASS_SOUNDS_METADATA_FILENAME), index_col=0)
df_amb = pd.read_csv(open(AMB_SOUNDS_METADATA_FILENAME), index_col=0)
df_source_lead = pd.read_csv(open(LEAD_DATAFRAME_SOURCE_FILENAME), index_col=0)
df_source_bass = pd.read_csv(open(BASS_DATAFRAME_SOURCE_FILENAME), index_col=0)
df_source_amb = pd.read_csv(open(AMB_DATAFRAME_SOURCE_FILENAME), index_col=0)
df_target = pd.read_csv(open(DATAFRAME_TARGET_FILE_FILENAME), index_col=0)

In [18]:
# Define some util functions
# NOTE: remember that if you update these util functions and want to do a new audio mosaicing, you'll need
# to re-run both this cell (to update the util functions) and the cell below (which uses the util functions to
# do the audio mosaicing).

loaded_audio_files = {}
import random

def get_audio_file_segment(file_path, start_sample, n_samples):
    """Load audio file. Try to get it from memory first. If not there, open it and save in memory for next time.
    """
    if file_path not in loaded_audio_files:
        loader = estd.MonoLoader(filename=file_path)
        audio = loader()
        loaded_audio_files[file_path] = audio
    else:
        audio = loaded_audio_files[file_path]

    # Return segment
    return audio[start_sample:start_sample + n_samples]

def find_similar_frames(query_frame, df_source_frames, n, features):
    """Find the 'n' mosr similar frames for a given 'query_frame' from those in the given 'df_source_frames'.
    Similarity is computed using a nearest neighbours algorithm and taking only into account the feature list
    given in the 'features' parameter.
    """
    query_frame = query_frame.reshape(1,-1)
    nbrs = NearestNeighbors(n_neighbors=n, algorithm='ball_tree').fit(df_source_frames[features].values)
    distances, indices = nbrs.kneighbors(query_frame)
    return [df_source_frames.iloc[k] for k in indices[0]]

def chose_frame_from_source_collection(target_frame, df_source_frames):
    """Choose one frame from 'df_source_frames' to replace the 'target_frame'.
    This implementation chooses the source frame usinng a similarity algorithm 'find_similar_frames',
    and a specific set of similarity features for timbre (MFCC).
    You can modify this function to implement new ways to choose a frame from the source.

    NOTE: 'target_frame' here should have the same features as 'df_source_frames' because both the
    target file and the source collection have been analyzed with the same analysis function.
    You can list available features using 'print(list(target_frame.keys()))' and print(list(df_source_frames.keys()))
    """
    n_neighbours_to_find = 10
    similarity_features = ['mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12']  # Use MFCCs for sound similarity ['mfcc_0', 'mfcc_1']

    # Find the 10 most similar frames to the target_frame from df_source_framesdf_source_units
    query_frame = target_frame[similarity_features].values
    similar_frames = find_similar_frames(query_frame, df_source_frames, n_neighbours_to_find, similarity_features)

    # Choose the first one as is the most similar
    most_similar_frame = random.choice(similar_frames)

    return most_similar_frame


In [19]:
# Do the reconstruction (audio mosaicing) of the target file using audio chunks (units, frames) from the sounds in the source collection

# Load target audio file to get its total length and to use it later
target_sound_filename = df_target.iloc[0]['path']
target_audio = estd.MonoLoader(filename=target_sound_filename)()
total_length_target_audio = len(target_audio)

# Init array where to put the audio of the reconstructed file
generated_audio = np.zeros(total_length_target_audio)

# Init list where to store IDs of sounds used in the reconstruction
selected_freesound_ids = []

# Iterate over the analyzed frame of the target file
print('Reconstructing audio file...')
for i in range(0, len(df_target)):
    target_frame = df_target.iloc[i]  # Get current frame

    # Choose one frame from the source collection to replace the target frame
    most_similar_frame = chose_frame_from_source_collection(target_frame, df_source)

    # Store freesound ID of the original sound where the 'most_similar_frame' belongs to
    selected_freesound_ids.append(most_similar_frame['freesound_id'])

    # Get the audio segment corresponding to the 'most_similar_frame'
    target_frame_n_samples = target_frame['end_sample'] - target_frame['start_sample']
    most_similar_frame_audio = get_audio_file_segment(most_similar_frame['path'], most_similar_frame['start_sample'], target_frame_n_samples)

    # Add audio segment to the reconstructed audio array
    generated_audio[target_frame['start_sample']:target_frame['start_sample']+len(most_similar_frame_audio)] = most_similar_frame_audio

# Store the results in a WAV file
generated_audio_filename = '{0}.reconstructed.wav'.format(target_sound_filename)
estd.MonoWriter(filename=generated_audio_filename, format='wav', sampleRate=44100)(essentia.array(generated_audio))
print('Audio generated and saved in {0}!\nIt contains audio from the following sounds:'.format(generated_audio_filename))
display(df.loc[df['freesound_id'].isin(selected_freesound_ids)])  # Show metadata for the Freesound sounds used in the reconstruction


Reconstructing audio file...
Audio generated and saved in 213524__garzul__120-bpm-distorded-drum-loop.wav.reconstructed.wav!
It contains audio from the following sounds:


,name,username,license,tags,freesound_id,path
8,Dog Bark,aunrea,http://creativecommons.org/publicdomain/zero/1.0/,"['pet', 'bark', 'dog']",495658,files/495658_7932944-hq.ogg
10,Single Dog Bark,kwahmah_02,http://creativecommons.org/publicdomain/zero/1.0/,"['animal', 'pet', 'dog', 'public-domain', 'woo...",277058,files/277058_4486188-hq.ogg
11,Midsized Dog Bark 02,noctaro,https://creativecommons.org/licenses/by/4.0/,"['barking', 'pet', 'bellen', 'dog', 'woof', 'a...",242403,files/242403_4165591-hq.ogg
13,Dog Bark Staffordshire Bullterrier,Anton,https://creativecommons.org/licenses/by/4.0/,"['sennheiser', 'sound-devices', 'dog', 'm-s', ...",157322,files/157322_58-hq.ogg
17,Barking Dog,SuperStudioBR,http://creativecommons.org/publicdomain/zero/1.0/,"['bark', 'happy', 'loud', 'dog', 'barking']",180977,files/180977_3370987-hq.ogg
18,Pomeranian Small Dog Barking.mp3,yunjish,http://creativecommons.org/publicdomain/zero/1.0/,"['barking', 'Pomeranian', 'dog', 'yappy', 'yip...",608732,files/608732_8729843-hq.ogg
24,Lighter Fire,RicardoRG,http://creativecommons.org/publicdomain/zero/1.0/,"['152', 'random', 'Percussion', 'Flint', 'Ligh...",463937,files/463937_9754348-hq.ogg
27,Velcro Rip,RicardoRG,http://creativecommons.org/licenses/by/3.0/,"['sound', '152', 'Drum', 'Garcia', 'Random', '...",463938,files/463938_9754348-hq.ogg
41,[Vocal] distorted lollievox vocal sample,waveplaySFX,http://creativecommons.org/publicdomain/zero/1.0/,"['webb', 'driven', 'vocal', 'indie', 'overdriv...",554949,files/554949_1676145-hq.ogg
43,Dj - www.hardsamples.com - by shock force.wav,shockforce,http://creativecommons.org/licenses/by/3.0/,"['acapella', 'acepellas', 'dj', 'gabber', 'har...",42213,files/42213_429662-hq.ogg


In [20]:
# Show further results of the reconstruction

# Plot waveforms
plt.figure(figsize=(15,5))
plt.plot(target_audio)
plt.axis([0, len(target_audio), -1, 1])
plt.title('Target audio')
plt.show()

plt.figure(figsize=(15,5))
plt.plot(generated_audio)
plt.axis([0, len(target_audio), -1, 1])
plt.title('Reconstructed')
plt.show()

# Show audio players
print('Target audio')
display(Audio(target_audio, rate=44100))

print('Reconstructed')
display(Audio(generated_audio, rate=44100))

print('Mix of both signals')
display(Audio(generated_audio * 0.5 + target_audio * 0.5, rate=44100))

Output hidden; open in https://colab.research.google.com to view.